## Load Plan2Scene Modules
__Re-run from here if you restart the runtime.__

In [5]:
# Load system modules
import zipfile
import io
import os.path as osp
import os
# from google.colab import files
import json
import pandas as pd
import sys
# from IPython.core.display import display, HTML
from IPython.display import display, HTML
from ipywidgets import interact
from PIL import ImageDraw, Image
import torch

# Load Plan2Scene modules
sys.path.append("./code/src")
from arch_parser.parser import parse_arch_json, parse_object_jsons, PreferredFormat, parse_scene_json_from_file
from plan2scene.common.residence import House
from plan2scene.config_manager import ConfigManager
from plan2scene.common.image_description import ImageDescription, ImageSource
from plan2scene.utils.io import load_image
from arch_parser.json_util import pil_to_data_url
from plan2scene.texture_gen.predictor import TextureGenPredictor
from plan2scene.texture_gen.utils.io import load_conf_eval
from plan2scene.crop_select.util import fill_textures
from plan2scene.texture_prop.utils import update_embeddings
from plan2scene.texture_prop.predictor import TexturePropPredictor
from plan2scene.texture_prop.graph_generators import InferenceHGG
from plan2scene.texture_prop.houses_dataset import HouseDataset
from torch_geometric.data import DataLoader
from plan2scene.common.house_parser import save_arch
from plan2scene.utils.tile_util import tile_image

# Initialize config manager
conf = ConfigManager()
conf.load_default_args()
conf.texture_gen.texture_synth_conf = "./conf/plan2scene/texture_synth_conf/v2.yml"

INFO:root:Using seed: 12415
INFO:root:Args: Namespace(seed=12415, data_paths='./conf/plan2scene/data_paths.json', house_gen='./conf/plan2scene/house_gen.json', metric_conf='./conf/plan2scene/metric.json', texture_gen='./conf/plan2scene/texture_gen.json', texture_prop='./conf/plan2scene/texture_prop_conf/default.json', render_config='./conf/plan2scene/render.json', seam_correct_config='./conf/plan2scene/seam_correct.json', labels_path='./conf/plan2scene/labels', log_level='INFO', drop='0.0', num_workers=4)


In [6]:
def upload():
  """
  Utility method for file uploads.
  """
  uploaded = files.upload()
  assert len(uploaded) == 1
  content = None
  for key in uploaded.keys():
    print('Uploaded file "{name}" with length {length} bytes'.format(
        name=key, length=len(uploaded[key])))
    return uploaded[key]

## Data
In this section, we will obtain the following files from you.
1. A floorplan vector in the scene.json format. Refer to the Rent3D++ dataset for examples. The scene.json format is [explained here](https://github.com/3dlg-hcvc/plan2scene/blob/main/docs/md/scene_json_format.md).
2. Rectified surface crops extracted from photos of the house.
  - If you wish to extract new rectified surface crops, [refer to instructions here](https://github.com/3dlg-hcvc/plan2scene/blob/main/docs/md/extract_crops.md).
  - Otherwise, you can provide rectified surface crops provided with the Rent3D++ dataset.
3. Photo room assignments specified in a photoroom.csv file. Refer to the Rent3D++ dataset for examples. The format of the photoroom.csv file is [described here](https://github.com/3dlg-hcvc/plan2scene/blob/main/docs/md/rent3dpp_data_organization.md).

__Task:__ Upload a scene.json file describing a floorplan vector.

In [ ]:
house_key = None
if not osp.exists("./data/processed/archs"):
  os.makedirs("./data/processed/archs")

scene_json_content = upload()
scene_json = json.loads(scene_json_content.decode())
scene_id = scene_json["scene"]["arch"]["id"]

with open(f"./data/processed/archs/{scene_id}.scene.json", "w") as f:
  f.write(scene_json_content.decode())

print("Sketch of floorplan vector.")
arch_house = parse_scene_json_from_file(f"./data/processed/archs/{scene_id}.scene.json",  None)
arch_house.sketch_house()